# Laboratorio sobre Agents y Tools

## Introducción

En este laboratorio práctico sobre Agentes y Tools explorarás cómo construir agentes inteligentes utilizando la librería LangChain y LangGraph.

A través de una serie de ejercicios prácticos, aprenderás a:

- Crear tu primer Agente ReAct.
- Integrar múltiples tools en un agente.
- Construir un chatbot con LangGraph.
- Agregar herramientas externas como Wikipedia y APIs de clima.
- Incorporar una base de conocimiento utilizando un índice de Pinecone.
- Implementar condiciones personalizadas para la selección de tools.

Este laboratorio está diseñado para ser un espacio de práctica donde podrás experimentar y aplicar los conceptos de agentes y tools en un entorno interactivo.

In [3]:
%pip install -U \
  langchain==0.3.7 \
  langchain-core==0.3.18 \
  langchain-openai==0.2.3 \
  langchain-community==0.3.7 \
  langchain-pinecone==0.2.0 \
  langchain-huggingface==0.1.2 \
  langgraph==0.2.19 \
  sentence-transformers \
  pinecone-client \
  pydantic==2.9.2 \
  pypdf

  Using cached langchain-0.3.7-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_core-0.3.18-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_openai-0.2.3-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_community-0.3.7-py3-none-any.whl.metadata (2.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.3-py3-none-any.whl.metadata (9.7 kB)
  Using cached pydantic_settings-2.11.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-text-splitters to determine which version is compatible with ot

  You can safely remove it manually.


## Parte 1: Creando tu primer ReAct Agent
Crea un agente ReAct que use un modelo de lenguaje (ChatOpenAI) y una tool para contar cuántas letras “r” hay en una palabra.
El agente debe recibir una pregunta del usuario, invocar la herramienta y responder con el resultado.

In [4]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
import os

# TODO: Inicializar el modelo de lenguaje gpt-4o-mini usando ChatOpenAI
api_key = os.environ.get("OPENAI_TOKEN")
model = ChatOpenAI(api_key=api_key, model="gpt-4o-mini")

# TODO: Define una tool que cuente las 'r' en una palabra
@tool
def count_r_in_word(word: str) -> int:
    """Count how many 'r' letters are in the given word."""
    return word.lower().count('r')

# TODO: Crear el agente con create_react_agent
app = create_react_agent(model=model, tools=[count_r_in_word])

# Pregunta del usuario
query = "How many r's are in the word 'Terrarium'?"

# TODO: Invocar el agente y mostrar la respuesta final
response = app.invoke({"messages": [("user", query)]})
print(response['messages'][-1].content)

There are 3 'r's in the word "Terrarium."


## Parte 2: Agregando múltiples tools
Amplía el agente para incluir una segunda herramienta que calcule el área de un rectángulo.
Luego, haz una consulta que combine ambas herramientas en una sola interacción.

In [5]:
# TODO: Define una tool que calcule el área de un rectángulo
@tool
def calculate_rectangle_area(length: float, width: float) -> float:
    """Calculate the area of a rectangle given its length and width."""
    return length * width

# TODO: Crear el agente con ambas tools
app = create_react_agent(model=model, tools=[count_r_in_word, calculate_rectangle_area])

query = "What is the area of a rectangle with length 5 and width 11? and how many r's are in 'Race'?"

# TODO: Invocar y mostrar la respuesta
response = app.invoke({"messages": [("user", query)]})
print(response['messages'][-1].content)

The area of the rectangle with a length of 5 and a width of 11 is 55 square units. Additionally, the word "Race" contains 1 letter 'r'.


## Parte 3: Construyendo un chatbot con LangGraph
Crea un StateGraph básico con un solo nodo chatbot que responda preguntas directamente usando el modelo LLM.

In [ ]:
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
import os

# Inicializa el modelo LLM desde variables de entorno
api_key = os.environ.get('OPENAI_TOKEN')
llm = ChatOpenAI(api_key=api_key, model="gpt-4o-mini")

# Crea el grafo y define el nodo chatbot
graph_builder = StateGraph(MessagesState)

def chatbot(state: MessagesState):
    return {"messages": [llm.invoke(state["messages"]) ]}

# Agrega nodos y bordes al grafo
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

# Invoca el grafo con una consulta
response = graph.invoke({"messages": [("human", "Who is Javier Milei?")]})
print(response["messages"][-1].content)

Ada Lovelace, born Augusta Ada Byron on December 10, 1815, is often regarded as one of the first computer programmers due to her work on Charles Babbage's early mechanical general-purpose computer, the Analytical Engine. She was the daughter of the poet Lord Byron and mathematician Annabella Milbanke.

Lovelace produced detailed notes on the Analytical Engine and is particularly known for her work on an algorithm intended to be processed by the machine, making her the first person to recognize that the machine had applications beyond mere calculation. Her vision of the machine being able to manipulate symbols and create art or music, for instance, was far ahead of her time.

Her contributions went largely unrecognized during her lifetime, but she has since become a symbol of women in technology and the importance of interdisciplinary thinking, blending mathematics, science, and the arts. Ada Lovelace died on November 27, 1852, but her legacy continues to inspire many in the fields of c

## Parte 4: Agregando WikipediaTool y usando stream_tool_responses
Integra la tool de Wikipedia al chatbot y usa la función stream_tool_responses para observar cómo el agente decide cuándo invocar la herramienta y cómo se entregan las respuestas en streaming.

In [ ]:
%pip install wikipedia

In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import MessagesState

# TODO: Inicializa el wrapper de Wikipedia
api_wrapper = __________
wikipedia_tool = __________

tools = [wikipedia_tool]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder = StateGraph(MessagesState)
graph_builder.add_node("chatbot", chatbot)

# TODO: Crea el nodo de herramientas y define las transiciones
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

def stream_tool_responses(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for item in event.values():
            print("Agent:", item["messages"])

query = "Who is Alan Turing?"
stream_tool_responses(query)

## Parte 5: Agregando una Tool de API externa
Agrega una tool que obtenga el clima actual de una ciudad usando la API de Open-Meteo.

Importante: en esta parte se puede utilizar cualquier otra API externa, la mencionada anteriormente es una sugerencia.

In [ ]:
import requests
from langchain_core.tools import tool

# TODO: Completar la función para obtener el clima actual
@tool
def weather_tool(city: str) -> str:
    """
    Retrieve current weather for a city using Open-Meteo.
    """
    # TODO: completar los pasos de geocodificación y consulta del clima
    geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}"
    geo_resp = __________
    # ...

# TODO: Agrega la nueva tool al grafo
tools = [wikipedia_tool, weather_tool]

# TODO: Volver a crear y compilar el grafo

## Parte 6: Agregando una base de conocimiento
Agrega una tool que consulte un índice de Pinecone con embeddings de un documento PDF.

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
import os

# TODO: Inicializar Pinecone y embeddings
os.environ["PINECONE_API_KEY"] = userdata.get("pinecone_api_key")
pc = Pinecone(api_key=userdata.get("pinecone_api_key"))
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Crear índice y vector store
index_name = "docs-index"
pc.create_index(name=index_name, dimension=384, metric="cosine",
                spec=ServerlessSpec(cloud="aws", region="us-east-1"))

vector_store = PineconeVectorStore.from_documents(chunks, embeddings, index_name=index_name)

@tool
def db_knowledge(query: str) -> str:
    """
    Retrieve relevant knowledge from the Pinecone vector store (LTM).
    """
    retriever = __________
    docs = retriever.get_relevant_documents(query)
    if not docs:
        return "No relevant knowledge found."
    return "\n".join([r.page_content for r in docs])

# TODO: Agregar tool, volver a construir el grafo y compilarlo

## Parte 7: Condiciones personalizadas para decisión de uso de tools
Crea un agente que decida entre las diferentes tools según el contenido del mensaje del usuario.
- Si el mensaje menciona "clima" → invocar weather_tool.
- Si menciona "wiki" → invocar wikipedia_tool.
- Si menciona "documento" → invocar db_knowledge.
- Para cualquier otro mensaje → invocar al LLM y terminar el flujo sin llamar a ninguna tool.

*Tener en cuenta que cada tool debe ser creada en un nodo independiente.

In [ ]:
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_core.messages import HumanMessage, AIMessage

# TODO: Crear nodos para el chatbot y las diferentes tools


# TODO: Definir condición personalizada para elegir tool
def tool_selector(state: MessagesState):
    last_message = state["messages"][-1]
    if not isinstance(last_message, HumanMessage):
        return END

    content = last_message.content.lower()

    return "tool_node_name"

graph_builder.add_conditional_edges(START, tool_selector, [])

# TODO: Agregar aristas faltantes necesarias

graph = graph_builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))